In [1]:
import pandas as pd
import numpy as np

In [2]:
school_csv = "Resources/schools_complete.csv"
student_csv = "Resources/students_complete.csv"

In [3]:
school_df = pd.read_csv(school_csv)
student_df = pd.read_csv(student_csv)

In [83]:
school_df = school_df.rename(columns={"name":"school"})
df = pd.merge(school_df, student_df, on="school")

In [84]:
min_passing_grade = 70

num_of_schools = df['School ID'].nunique()
num_of_students = df['Student ID'].nunique()
tot_school_budget = school_df['budget'].sum()
avg_math_score_tot = df['math_score'].mean()
avg_reading_score_tot = df['reading_score'].mean()
percent_passing_math_tot = df.loc[df['math_score'] >= min_passing_grade, :]['math_score'].count() / num_of_students
percent_passing_reading_tot = df.loc[df['reading_score'] >= min_passing_grade, :]['reading_score'].count() / num_of_students
overall_passing_score_tot = (percent_passing_math_tot + percent_passing_reading_tot) / 2

district_summary_df = pd.DataFrame({"Total Schools":[num_of_schools], "Total Students":[num_of_students], "Total Budget":[tot_school_budget], "Average Math Score":[avg_math_score_tot], "Average Reading Score":[avg_reading_score_tot], "% Passing Math":[percent_passing_math_tot], "% Passing Reading":[percent_passing_reading_tot], "% Overall Passing Rate":[overall_passing_score_tot]})
district_summary_df

,% Overall Passing Rate,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Total Budget,Total Schools,Total Students
0,0.803932,0.749809,0.858055,78.985371,81.87784,24649428,15,39170


In [86]:
grouped_schools_funcs = df.groupby(['school']).agg({'Student ID':['count'], 'reading_score':['mean'], "math_score":['mean']})
grouped_schools_funcs = grouped_schools_funcs.reset_index()
grouped_schools_funcs.columns = ['School Name', 'Number of Students', 'Avg Reading Score', 'Avg Math Score']

short_school_df = school_df[['school', 'type', 'budget']]
short_school_df.columns = ['School Name', 'School Type', 'Budget']
school_summary_df = pd.merge(short_school_df, grouped_schools_funcs, on='School Name')
school_summary_df['Per Student Budget'] = school_summary_df['Budget'] / school_summary_df['Number of Students']

df_passing_reading = df.loc[df['reading_score'] >= min_passing_grade, :]
df_passing_reading_by_school = df_passing_reading.groupby('school')
percent_passing_reading_by_school = df_passing_reading_by_school['reading_score'].count() / df.groupby('school')['reading_score'].count()
percent_passing_reading_by_school_df = percent_passing_reading_by_school.reset_index()
percent_passing_reading_by_school_df.columns = ['School Name', '% Passing Reading']

df_passing_math = df.loc[df['math_score'] >= min_passing_grade, :]
df_passing_math_by_school = df_passing_math.groupby('school')
percent_passing_math_by_school = df_passing_math_by_school['math_score'].count() / df.groupby('school')['math_score'].count()
percent_passing_math_by_school_df = percent_passing_math_by_school.reset_index()
percent_passing_math_by_school_df.columns = ['School Name', '% Passing Math']

percent_passing_by_school_df = pd.merge(percent_passing_reading_by_school_df, percent_passing_math_by_school_df, on="School Name")
percent_passing_by_school_df
merged_school_summary_df = pd.merge(school_summary_df, percent_passing_by_school_df, on="School Name")
merged_school_summary_df['% Overall Passing Rate'] = (merged_school_summary_df['% Passing Reading'] + merged_school_summary_df['% Passing Math']) / 2
merged_school_summary_df

,School Name,School Type,Budget,Number of Students,Avg Reading Score,Avg Math Score,Per Student Budget,% Passing Reading,% Passing Math,% Overall Passing Rate
0,Huang High School,District,1910635,2917,81.182722,76.629414,655.0,0.813164,0.656839,0.735002
1,Figueroa High School,District,1884411,2949,81.158020,76.711767,639.0,0.807392,0.659885,0.733639
2,Shelton High School,Charter,1056600,1761,83.725724,83.359455,600.0,0.958546,0.938671,0.948609
3,Hernandez High School,District,3022020,4635,80.934412,77.289752,652.0,0.808630,0.667530,0.738080
4,Griffin High School,Charter,917500,1468,83.816757,83.351499,625.0,0.971390,0.933924,0.952657
5,Wilson High School,Charter,1319574,2283,83.989488,83.274201,578.0,0.965396,0.938677,0.952037
6,Cabrera High School,Charter,1081356,1858,83.975780,83.061895,582.0,0.970398,0.941335,0.955867
7,Bailey High School,District,3124928,4976,81.033963,77.048432,628.0,0.819333,0.666801,0.743067
8,Holden High School,Charter,248087,427,83.814988,83.803279,581.0,0.962529,0.925059,0.943794
9,Pena High School,Charter,585858,962,84.044699,83.839917,609.0,0.959459,0.945946,0.952703


In [88]:
top_schools = merged_school_summary_df.sort_values('% Overall Passing Rate', ascending=False).head()
top_schools

,School Name,School Type,Budget,Number of Students,Avg Reading Score,Avg Math Score,Per Student Budget,% Passing Reading,% Passing Math,% Overall Passing Rate
6,Cabrera High School,Charter,1081356,1858,83.975780,83.061895,582.0,0.970398,0.941335,0.955867
14,Thomas High School,Charter,1043130,1635,83.848930,83.418349,638.0,0.973089,0.932722,0.952905
9,Pena High School,Charter,585858,962,84.044699,83.839917,609.0,0.959459,0.945946,0.952703
4,Griffin High School,Charter,917500,1468,83.816757,83.351499,625.0,0.971390,0.933924,0.952657
5,Wilson High School,Charter,1319574,2283,83.989488,83.274201,578.0,0.965396,0.938677,0.952037


In [89]:
bottom_schools = merged_school_summary_df.sort_values('% Overall Passing Rate').head()
bottom_schools

,School Name,School Type,Budget,Number of Students,Avg Reading Score,Avg Math Score,Per Student Budget,% Passing Reading,% Passing Math,% Overall Passing Rate
11,Rodriguez High School,District,2547363,3999,80.744686,76.842711,637.0,0.802201,0.663666,0.732933
1,Figueroa High School,District,1884411,2949,81.158020,76.711767,639.0,0.807392,0.659885,0.733639
0,Huang High School,District,1910635,2917,81.182722,76.629414,655.0,0.813164,0.656839,0.735002
12,Johnson High School,District,3094650,4761,80.966394,77.072464,650.0,0.812224,0.660576,0.736400
13,Ford High School,District,1763916,2739,80.746258,77.102592,644.0,0.792990,0.683096,0.738043
